In [1]:
from google.colab import files
uploaded = files.upload()


Saving task1_cleaned.csv to task1_cleaned.csv


In [2]:
import pandas as pd

df = pd.read_csv("task1_cleaned.csv")
df.head()


/tmp/ipython-input-1007699405.py:3: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("task1_cleaned.csv")


,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims,LossRatio
0,145249,12827,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0,0.0
1,145249,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0,0.0
2,145249,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0,NaN
3,145255,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0,0.0
4,145255,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0,NaN


In [3]:
# Claim frequency: proportion of policies with at least one claim
df["HasClaim"] = df["TotalClaims"] > 0

claim_frequency = df.groupby("Province")["HasClaim"].mean()
claim_frequency


,HasClaim
Province,
Eastern Cape,0.001648
Free State,0.001358
Gauteng,0.003356
KwaZulu-Natal,0.002845
Limpopo,0.002698
Mpumalanga,0.002428
North West,0.002436
Northern Cape,0.001254
Western Cape,0.002166


In [4]:
# Claim severity: average claim amount given a claim occurred
claim_severity = df[df["HasClaim"]]["TotalClaims"].groupby(df["Province"]).mean()
claim_severity


,TotalClaims
Province,
Eastern Cape,27128.533277
Free State,32265.661085
Gauteng,22243.878396
KwaZulu-Natal,29609.487473
Limpopo,15171.294187
Mpumalanga,15979.553421
North West,16963.467035
Northern Cape,11186.313596
Western Cape,28095.849881


In [5]:
from scipy import stats

# Prepare data: list of LossRatio arrays by Province
groups = [group["LossRatio"].dropna() for name, group in df.groupby("Province")]

# ANOVA test
f_stat, p_value = stats.f_oneway(*groups)
print(f"ANOVA F-statistic: {f_stat:.3f}, p-value: {p_value:.4f}")

if p_value < 0.05:
    print("Reject null hypothesis → Risk differs across Provinces")
else:
    print("Fail to reject null → No significant risk difference across Provinces")


ANOVA F-statistic: nan, p-value: nan
Fail to reject null → No significant risk difference across Provinces


/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:127: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)


In [6]:
male_loss = df[df["Gender"]=="Male"]["LossRatio"].dropna()
female_loss = df[df["Gender"]=="Female"]["LossRatio"].dropna()

t_stat, p_val = stats.ttest_ind(male_loss, female_loss, equal_var=False)
print(f"T-test statistic: {t_stat:.3f}, p-value: {p_val:.4f}")

if p_val < 0.05:
    print("Reject null hypothesis → Risk differs between Men and Women")
else:
    print("Fail to reject null → No significant risk difference between Men and Women")


T-test statistic: nan, p-value: nan
Fail to reject null → No significant risk difference between Men and Women


/usr/local/lib/python3.12/dist-packages/scipy/stats/_stats_py.py:6277: RuntimeWarning: invalid value encountered in scalar subtract
  d = mean1 - mean2
/usr/local/lib/python3.12/dist-packages/scipy/stats/_stats_py.py:6831: RuntimeWarning: invalid value encountered in scalar subtract
  estimate = m1 - m2


In [7]:
df["Margin"] = df["TotalPremium"] - df["TotalClaims"]

groups_zip = [group["Margin"].dropna() for name, group in df.groupby("PostalCode")]
f_stat_zip, p_val_zip = stats.f_oneway(*groups_zip)

print(f"ANOVA F-statistic: {f_stat_zip:.3f}, p-value: {p_val_zip:.4f}")

if p_val_zip < 0.05:
    print("Reject null hypothesis → Margin differs across zip codes")
else:
    print("Fail to reject null → No significant margin difference between zip codes")


ANOVA F-statistic: 0.871, p-value: 0.9977
Fail to reject null → No significant margin difference between zip codes
